In [ ]:
%pip install matplotlib
%pip install geopandas

import os
import geopandas as gpd
import matplotlib.pyplot as plt

# Path to your shapefile (update filename as needed)
shapefile_path = "data/boundaries/uga_admbnda_adm2_ubos_20200824.shp"

# Check file exists and help debugging if not
if not os.path.exists(shapefile_path):
    print(f"Shapefile not found at {shapefile_path}.")
    base_dir = os.path.dirname(shapefile_path)
    if os.path.isdir(base_dir):
        print('Files in', base_dir, ':', os.listdir(base_dir))
    raise FileNotFoundError(shapefile_path)

# Load shapefile
try:
    gdf = gpd.read_file(shapefile_path)
except Exception as e:
    raise RuntimeError(f"Failed to read shapefile: {e}")

# Basic exploration
print('Rows, columns:', gdf.shape)
print('Columns:', list(gdf.columns))
print('CRS:', gdf.crs)
print('Total bounds:', gdf.total_bounds)

print('\nPreview:' )
print(gdf.head())

# Plot all boundaries
ax = gdf.plot(figsize=(10, 10), color='lightgrey', edgecolor='black')
ax.set_title('Uganda Administrative Boundaries (Level 2)')
plt.show()

# Find a column that contains 'Kampala' values
name_col = None
for col in gdf.select_dtypes(include=['object']).columns:
    if gdf[col].astype(str).str.contains('Kampala', case=False, na=False).any():
        name_col = col
        break

kampala = None
if name_col is None:
    # try scanning all columns as strings
    mask = None
    for col in gdf.columns:
        try:
            s = gdf[col].astype(str).str.contains('Kampala', case=False, na=False)
            if mask is None:
                mask = s
            else:
                mask = mask | s
        except Exception:
            pass
    if mask is not None and mask.any():
        kampala = gdf[mask]
        print('Found Kampala by scanning all columns')
    else:
        print("Could not find 'Kampala' in any column. Sample unique values from first string columns:")
        for col in gdf.select_dtypes(include=['object']).columns[:5]:
            print(col, '->', gdf[col].dropna().unique()[:10])
else:
    kampala = gdf[gdf[name_col].astype(str).str.contains('Kampala', case=False, na=False)]
    print(f"Filtering by column: {name_col}")

# Show and plot Kampala if found
if kampala is not None and len(kampala):
    print('Kampala rows:', len(kampala))
    print(kampala)
    ax = gdf.plot(figsize=(8, 8), color='lightgrey', edgecolor='black')
    kampala.plot(ax=ax, color='orange', edgecolor='black')
    ax.set_title('Kampala District (highlighted)')
    minx, miny, maxx, maxy = kampala.total_bounds
    xpad = (maxx - minx) * 0.1
    ypad = (maxy - miny) * 0.1
    ax.set_xlim(minx - xpad, maxx + xpad)
    ax.set_ylim(miny - ypad, maxy + ypad)
    plt.show()
else:
    print('No Kampala geometry to plot')

## Setup: geopandas installation (recommended via conda)

If you don't have geopandas installed or pip install fails on Windows, the easiest and most reliable route is conda (Miniconda/Anaconda):

```
# Create an environment with geopandas (recommended)
conda create -n geo -c conda-forge geopandas
conda activate geo
```

If you must use pip, try installing prebuilt wheels or use the `--find-links` option, but conda-forge avoids most Windows build issues.


In [ ]:
# Quick runtime check: attempt to import geopandas and show versions
try:
	import geopandas as gpd
	import sys
	print('geopandas version:', getattr(gpd, '__version__', 'unknown'))
	print('python version:', sys.version)
except Exception as e:
	print('Failed to import geopandas:', e)